In [ ]:
import pandas as pd
import yaml

from zephir.methods import build_annotations
from zephir.utils.utils import *

%matplotlib inline
from tqdm.notebook import tqdm

In [ ]:
# path to Zephir dataset
dataset = Path('.')

# path to DeepLabCut project directory
path_to_dlc_project = Path('.')

In [ ]:
# creating directories for DeepLabCut project
# only run this if it does not yet exist
Path.mkdir(path_to_dlc_project)
Path.mkdir(path_to_dlc_project / 'labeled-data')
Path.mkdir(path_to_dlc_project / 'labeled-data' / 'zeir')

In [ ]:
metadata = get_metadata(dataset)
img_shape = np.array([metadata['shape_x'], metadata['shape_y']])

# retrieving Zephir user arguments from checkpoint.pt
# edit the dictionary before the next cell as needed
args = get_checkpoint(dataset, 'args', verbose=True)
# args['--wlid_ref'] = None
# args['--n_ref'] = None
# update_checkpoint(dataset,{'args': args})

# retrieving Zephir variable container from checkpoint.pt
# update the values in the container before the next cell as needed
container = get_checkpoint(dataset, 'container')
# container.update({
#     'exclude_self': False,
#     'exclusive_prov': None,
# })

print(f'Variables for build_annotations in container:\n'
      f'\texclude_self: {container.get("exclude_self")}\n'
      f'\texclusive_prov: {container.get("exclusive_prov")}\n'
      f'\tshape_t: {container.get("shape_t")}')

args

In [ ]:
# define the frames to use for training DeepLabCut
t_list = []
# t_list = eval(args['--t_ref']) if args['--t_ref'] else None

container, results = build_annotations(
    container=container,
    annotation=None,
    t_ref=t_list,
    wlid_ref=eval(args['--wlid_ref']) if args['--wlid_ref'] else None,
    n_ref=int(args['--n_ref']) if args['--n_ref'] else None,
)

annot = np.array(container.get('annot'))
shape_n = container.get('shape_n')
t_annot = container.get('t_annot')
worldline_id = container.get('worldline_id')

In [ ]:
path_to_dir = path_to_dlc_project / 'labeled-data' / 'zeir'

# creates a folder of images as training input
img_address_list = []
for t in t_annot:
    frame = get_slice(dataset, t)
    frame = np.transpose(frame[:, 0, ...], (1, 2, 0))
    frame = np.append(frame, frame[:, :, 0, None], axis=-1)
    address = str(path_to_dir / f'img{t:04}.png')
    cv2.imwrite(address, frame)
    img_address_list.append(address)

In [ ]:
# compiling a list of body parts to track
# define an explicit list of parts directly, must be in the same order as worldlines
# use 'dlc' to import the list of parts from existing DeepLabCut project
# use 'zeir' to import the list from worldlines.h5, created in annotator GUI
bodyparts = None

if type(bodyparts) is list:
    pass

elif bodyparts == 'dlc':
    with open(str(path_to_dlc_project / 'config.yaml'), 'r') as cfg:
        config = yaml.safe_load(cfg)
    bodyparts = config['bodyparts']

elif bodyparts == 'zeir' and (dataset / 'worldlines.h5').is_file():
    with h5py.File(dataset / 'worldlines.h5', 'r') as f:
        worldlines = pd.DataFrame()
        for k in f:
            worldlines[k] = f[k]
    bodyparts = np.array(worldlines["name"])[worldline_id]

else:
    bodyparts = [str(w) for w in worldline_id]

bodyparts

In [ ]:
# compiling the annotations pulled from annotations.h5 to a DeepLabCut-compatible dataframe
df = None
for i in tqdm(range(shape_n), desc='Exporting annotations', unit='bodypart'):
    bodypart = bodyparts[i]
    index = pd.MultiIndex.from_product(
        [['zeir'], [bodypart], ['x', 'y']],
        names=['scorer', 'bodyparts', 'coords']
    )

    xyz_scaled = (annot[:, i, :2] + 1) / 2 * img_shape
    xlist, ylist = xyz_scaled[:, 0], xyz_scaled[:, 1]
    _df = pd.DataFrame(
        np.vstack([xlist, ylist]).T,
        columns=index,
        index=img_address_list
    )
    if df is None:
        df = _df
    else:
        df = pd.concat(
            [df, _df],
            axis=1
        )

# saving dataframe to file
df.to_csv(str(path_to_dir / 'CollectedData_zeir.csv'))
df.to_hdf(
    str(path_to_dir / 'CollectedData_zeir.h5'),
    'df_with_missing',
    format='table',
    mode='w'
)